In [2]:
import pandas as pd
import numpy as np
import math
import io
from google.colab import files
uploaded=files.upload()
bt_coin=pd.read_csv(io.BytesIO(uploaded['Bitcoin.csv']))
bt_coin.head()

Saving Bitcoin.csv to Bitcoin.csv


,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume
0,31-05-2019,01-06-2019 00:00,bitstamp,btc_eur,1996.72,2029.99,-1.638924,2005.50,2005.56,1950.00,2063.73,2314.500750
1,31-05-2019,01-06-2019 00:00,bitflyer,btc_jpy,267098.00,269649.00,-0.946045,267124.00,267267.00,267124.00,267267.00,70922.880110
2,31-05-2019,01-06-2019 00:00,korbit,btc_krw,3003500.00,3140000.00,-4.347134,3003500.00,3004000.00,3002000.00,3209500.00,6109.752872
3,31-05-2019,01-06-2019 00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282020
4,31-05-2019,01-06-2019 00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516


In [3]:
#for scaling, metrics and tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [8]:
#for building LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [5]:
#for visualizing the prediction
from bokeh.plotting import figure
from bokeh.io import output_notebook,show,push_notebook
from bokeh.models import Legend

In [9]:
#counting the total number of cryptocurrencies
bt_coin['rpt_key'].value_counts()
#keeping only USD for our prediction
bt_coin1 = bt_coin.loc[(bt_coin['rpt_key'] =='btc_usd')]
bt_coin1.head()

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume
3,31-05-2019,01-06-2019 00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282020
4,31-05-2019,01-06-2019 00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516
15,01-06-2019,01-06-2019 00:01,bitstamp,btc_usd,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698600
16,01-06-2019,01-06-2019 00:01,okcoin,btc_usd,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516
23,01-06-2019,01-06-2019 00:02,bitstamp,btc_usd,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110910


In [10]:
#setting a certain threshold date above which we start predicting
bt_coin1 = bt_coin1.reset_index(drop=True)
bt_coin1['datetime']=pd.to_datetime(bt_coin1['datetime_id'])
bt_coin1 = bt_coin1.loc[bt_coin1['datetime']>pd.to_datetime('2019-06-28 00:00:00')]

In [11]:
#keeping only the required columns for prediction
bt_coin1=bt_coin1[['datetime','last','low','high','volume']] #keeping only these values
bt_coin1.head()

,datetime,last,low,high,volume
17280,2019-07-06 00:00:00,2820.00,2608.25,2933.0,21078.15618
17281,2019-07-06 00:00:00,2940.32,2684.28,3095.0,4019.75600
17282,2019-07-06 00:01:00,2820.00,2608.25,2933.0,21038.90547
17283,2019-07-06 00:01:00,2946.00,2684.28,3095.0,4018.37100
17284,2019-07-06 00:02:00,2819.99,2608.25,2933.0,21035.46284


In [12]:
bt_coin2 = bt_coin1['last']
bt_coin2

17280     2820.00
17281     2940.32
17282     2820.00
17283     2946.00
17284     2819.99
           ...   
125433    2394.03
125434    2320.47
125435    2394.03
125436    2320.47
125437    2394.03
Name: last, Length: 47676, dtype: float64

In [13]:
#Scaling the features ranging from one to two
scaler = MinMaxScaler(feature_range=(1,2))
bt_coin2 = scaler.fit_transform(np.array(bt_coin2).reshape(-1,1))

In [14]:
bt_coin2

array([[1.61532178],
       [1.74297142],
       [1.61532178],
       ...,
       [1.16340258],
       [1.08536145],
       [1.16340258]])

In [15]:
out = [] #storing the array values in a dummy variable
for i in bt_coin2:
  out.append(i[0])
bt_coin2=out

In [16]:
bt_coin2

[1.615321776400942,
 1.742971418871608,
 1.615321776400942,
 1.7489974325786668,
 1.61531116722188,
 1.7498567760826664,
 1.6194805745931378,
 1.7592671179104156,
 1.6244881071102713,
 1.7639457658766364,
 1.6252625771817772,
 1.7833181268433447,
 1.6291137091811834,
 1.78802860234675,
 1.6362961234059705,
 1.7801141547667036,
 1.6312567633516517,
 1.774459462326805,
 1.6348214475163911,
 1.767033036983598,
 1.6365507437034523,
 1.767033036983598,
 1.6381527297417722,
 1.7666298881792524,
 1.6312355449935283,
 1.7610918967090323,
 1.62909249082306,
 1.756370812026565,
 1.620626365931804,
 1.7501750514545185,
 1.6248700375564935,
 1.7501007872010863,
 1.6344395170701689,
 1.7616223556621189,
 1.627426849710369,
 1.7616223556621189,
 1.634651700651403,
 1.7656220161683889,
 1.6419614250249315,
 1.765579579452142,
 1.6459610855312015,
 1.7649430287084384,
 1.6482314498504103,
 1.765611406989327,
 1.6444864096416216,
 1.765611406989327,
 1.6481890131341634,
 1.7672452205648321,
 1.64822084

In [17]:
#Splitting the dataset into 60-40 for testing and training
train_size=int(len(bt_coin2)*0.6)
test_size=len(bt_coin2) - train_size
train, test = bt_coin2[0:train_size],bt_coin2[train_size:len(bt_coin2)]
print(len(train), len(test))

28605 19071


In [18]:
#convert an array of values into a dataset matrix
def create_dataset(dataset,look_back):
 dataX,dataY=[],[]
 for i in range(len(dataset)-look_back-1):
   a=dataset[i:(i+look_back)]
   dataX.append(a)
   dataY.append(dataset[i+look_back])
 return np.array(dataX),np.array(dataY)

In [19]:
look_back=10
trainX,trainY = create_dataset(train,look_back)
testX,testY =  create_dataset(test,look_back)

In [20]:
trainX

array([[1.61532178, 1.74297142, 1.61532178, ..., 1.75926712, 1.62448811,
        1.76394577],
       [1.74297142, 1.61532178, 1.74899743, ..., 1.62448811, 1.76394577,
        1.62526258],
       [1.61532178, 1.74899743, 1.61531117, ..., 1.76394577, 1.62526258,
        1.78331813],
       ...,
       [1.36827643, 1.30647796, 1.36827643, ..., 1.30588385, 1.36827643,
        1.30659467],
       [1.30647796, 1.36827643, 1.30248891, ..., 1.36827643, 1.30659467,
        1.36827643],
       [1.36827643, 1.30248891, 1.36827643, ..., 1.30659467, 1.36827643,
        1.30659467]])

In [21]:
trainY

array([1.62526258, 1.78331813, 1.62911371, ..., 1.36827643, 1.30659467,
       1.36827643])

In [22]:
testX

array([[1.36827643, 1.30658406, 1.36827643, ..., 1.30525791, 1.37142736,
        1.30298754],
       [1.30658406, 1.36827643, 1.30658406, ..., 1.37142736, 1.30298754,
        1.38343695],
       [1.36827643, 1.30658406, 1.37142736, ..., 1.30298754, 1.38343695,
        1.29708884],
       ...,
       [1.08755755, 1.16051688, 1.08755755, ..., 1.17580471, 1.08756816,
        1.16340258],
       [1.16051688, 1.08755755, 1.17580471, ..., 1.08756816, 1.16340258,
        1.08536145],
       [1.08755755, 1.17580471, 1.08509623, ..., 1.16340258, 1.08536145,
        1.16340258]])

In [23]:
testY

array([1.38343695, 1.29708884, 1.38342634, ..., 1.08536145, 1.16340258,
       1.08536145])

In [24]:
#reshape input to be[samples,time steps,features]
trainX=np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX=np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

In [25]:
trainX.shape

(28594, 1, 10)

In [26]:
# Building the LSTM model

In [27]:
model=Sequential()
model.add(LSTM(4,input_shape=(1,look_back)))#four hidden layers
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(trainX,trainY,epochs=100,batch_size=256,verbose=1)

Epoch 1/100
112/112 [==============================] - 4s 7ms/step - loss: 1.6970
Epoch 2/100
112/112 [==============================] - 0s 4ms/step - loss: 0.1712
Epoch 3/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0208
Epoch 4/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0178
Epoch 5/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0167
Epoch 6/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0153
Epoch 7/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0136
Epoch 8/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0106
Epoch 9/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0077
Epoch 10/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0055
Epoch 11/100
112/112 [==============================] - 0s 4ms/step - loss: 0.0039
Epoch 12/100
112/112 [==============================] - 1s 6ms/step - loss: 0.0028
Epoch 13/100


In [28]:
# Predicting the model

In [29]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict=scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

596/596 [==============================] - 1s 2ms/step


In [30]:
# Finding out the RMSE value

In [31]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' %(trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' %(testScore))

Train Score: 8.87 RMSE
Test Score: 8.58 RMSE


In [32]:
#shift train predictions for plotting
trainPredictPlot=np.zeros(len(bt_coin2))
trainPredictPlot[:]=np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back]=trainPredict[:,0]

In [33]:
#shift test predictions for plotting
testPredictPlot=np.zeros(len(bt_coin2),dtype=np.float64)
testPredictPlot[:]=np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(bt_coin2)-1] = testPredict[:,0]

In [36]:
output_notebook()

#prediction plot
p=figure(width=900,height=300)
p.line(np.arange(len(bt_coin1['last'])),bt_coin1['last'],legend_label='Actual',color='Black')
tpp=pd.DataFrame(trainPredictPlot,columns=["close"],index=bt_coin1.index).close
qpp=pd.DataFrame(testPredictPlot,columns=["close"],index=bt_coin1.index).close
p.line(np.arange(len(tpp)),tpp,legend_label='Training',color='blue')
p.line(np.arange(len(qpp)),qpp,legend_label='Testing',color='orange')
p.legend.location="top_right"
show(p)